<a href="https://colab.research.google.com/github/juliosoterooo/Extracci-n-de-datos/blob/main/Act%203.7%20Despliegue%20de%20Datos/dashboard_todo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalamos la libreria de STREAMLIT
%pip install streamlit

In [2]:
#Instalamos la libreria de PLOTLY
%pip install plotly

In [3]:
#Instalamos librería para análisis de variables categóricas
%pip install funpymodeling

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.

In [4]:
#Instalamos el tunnel local (node.js)
#Para instalar npm en visual studio
#1.Desde Google escribir node.js
#2. Instalar la versión más recomendada
!npm install localtunnel


added 22 packages, and audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [6]:
#Cargar archivo csv desde equipo
from google.colab import files

uploaded = files.upload()

Saving Datos_Limpios_iqr(méxico).csv to Datos_Limpios_iqr(méxico).csv


In [9]:
%%writefile app.py
# Creamos el archivo de la APP en el intérprete principal (Python)

############################# IMPLEMENTACIÓN DE DASHBOARD ################################

# Verificamos que todas las librerías se puedan importar
# Importamos librerías necesarias para el dashboard y análisis de datos




# Importamos librerías adicionales para análisis estadístico y modelado
import pandas as pd
import streamlit as st
from funpymodeling.exploratory import freq_tbl
import plotly.express as px
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score

#################################################################

# Definimos la instancia de Streamlit con caché para optimizar la carga de recursos
@st.cache_resource
def load_data_mexico():

    # Lectura del archivo CSV
    df1 = pd.read_csv("Datos_Limpios_iqr(méxico).csv")
    df1.drop(columns='Unnamed: 0', inplace=True)

    # Crear una copia del DataFrame original
    data_copia = df1.copy()

    # Reemplazar datos categóricos por numéricos en la columna 'room_type'
    data_copia['room_type'] = data_copia['room_type'].replace({
        'Entire home/apt': 1,
        'Hotel room': 2,
        'Private room': 3,
        'Shared room': 4
    }).astype(int)

    # Seleccionar solo columnas numéricas para el análisis
    data_evaluar = data_copia.select_dtypes(['float', 'int'])  # Devuelve Columnas

    # Calcular la matriz de correlación entre variables
    corr_factors = data_evaluar.corr()
    # Obtener el valor absoluto de las correlaciones
    corr_factors1 = abs(corr_factors)

    # Obtener los nombres de las columnas numéricas
    numeric_cols1 = data_evaluar.columns

    return df1, data_copia, data_evaluar, numeric_cols1, corr_factors1

#################################################################
# Cargar los datos utilizando la función definida anteriormente
df1, data_copia, data_evaluar, numeric_cols1, corr_factors1 = load_data_mexico()

#################################################################

# PAGINA PRINCIPAL - Título y encabezado
st.title('AIRBNB-CDMX')
st.header('Análisis de Datos')

# Generamos un cuadro de selección (Checkbox) en una barra lateral para mostrar correlaciones lineales
check_box = st.sidebar.checkbox(label="Mostrar Correlaciones Lineales")

if check_box:
    # Mostramos la matriz de correlaciones
    st.write(corr_factors1)

# Generamos un cuadro de selección (Checkbox) en una barra lateral para mostrar el Heat Map
check_map = st.sidebar.checkbox(label="Mostrar Heat Map")

if check_map:
    # Creamos y mostramos el Heat Map de las correlaciones
    heatmap = px.imshow(corr_factors1)
    st.plotly_chart(heatmap)

###################### CREACIÓN DEL DASHBOARD ################################

# 1. CREACIÓN DE LA SIDEBAR
# Generamos los encabezados para la barra lateral (sidebar)
st.sidebar.title("Menú Principal")
st.sidebar.header("Panel de Selección")

#################################################################

# 2. CREACIÓN DE LOS FRAMES
# Generamos los Frames que utilizaremos en el diseño del dashboard
# Widget 1: Selectbox para elegir entre diferentes variables y análisis
Variables = st.selectbox(
    label="Selecciona una Variable",
    options=[
        "Página de Inicio",
        "Análisis Univariado",
        "Regresión Simple",
        "Regresión Múltiple",
        "Regresión No Lineal",
        "Regresión Logística",
        "ANOVA"
    ]
)

#################################################################

# 3. FRAME: Página de Inicio
if Variables == "Página de Inicio":

    # Mostrar una imagen de la Ciudad de México
    st.image("https://www.civitatis.com/f/mexico/ciudad-de-mexico/autobus-turistico-mexico-589x392.jpg",
             caption="Ciudad de México", use_column_width=True)

    # Título de bienvenida
    st.title("¡Bienvenidos a mi Dashboard!")

    # Texto de bienvenida
    st.write("Aquí encontraremos información sobre los Airbnbs en Ciudad de México y algunos hallazgos y resultados.")

    # Botones para navegar a las diferentes áreas del dashboard
    if st.button('Análisis Univariado'):
        Variables = "Análisis Univariado"

    if st.button('Regresión Simple'):
        Variables = "Regresión Simple"

    if st.button('Regresión Múltiple'):
        Variables = "Regresión Múltiple"

    if st.button('Regresión No Lineal'):
        Variables = "Regresión No Lineal"

    if st.button('Regresión Logística'):
        Variables = "Regresión Logística"

    if st.button('ANOVA'):
        Variables = "ANOVA"

#################################################################

# 4. FRAME: Análisis Univariado
if Variables == "Análisis Univariado":
    st.title("Análisis Univariado")
    st.write("""
        En esta sección puedes explorar la distribución de una variable a la vez.
        Selecciona la variable que deseas analizar y visualiza su distribución, estadísticas descriptivas y más.
    """)

    # Identificar variables categóricas y numéricas disponibles
    categorical_vars = [col for col in data_copia.select_dtypes(include=['object', 'category']).columns]
    numerical_vars = data_copia.select_dtypes(include=['int64', 'float64']).columns.tolist()
    numerical_vars = [var for var in numerical_vars if var not in categorical_vars]

    # Widget para seleccionar el tipo de variable a analizar
    var_type = st.radio(
        "Selecciona el Tipo de Variable a Analizar:",
        ("Categoría", "Numérica"),
        key="univariate_var_type"
    )

    if var_type == "Categoría":
        selected_var = st.selectbox(
            "Selecciona la Variable Categórica:",
            options=categorical_vars,
            key="univariate_cat_var"
        )

        if selected_var:
            st.write(f"### Distribución de `{selected_var}`")
            count_data = df1[selected_var].value_counts().reset_index()
            count_data.columns = [selected_var, 'count']

            # Mostrar la tabla de conteo
            st.dataframe(count_data)

            # Gráfico de barras
            fig = px.bar(count_data, x=selected_var, y='count', title=f'Conteo de `{selected_var}`')
            st.plotly_chart(fig)

            # Agregar gráfico de pastel
            fig_pie = px.pie(count_data, names=selected_var, values='count', title=f'Distribución de `{selected_var}` en Gráfico de Pastel')
            st.plotly_chart(fig_pie)

            # Estadísticas descriptivas
            st.write("### Estadísticas Descriptivas")
            st.write(df1[selected_var].describe())

    elif var_type == "Numérica":
        selected_var = st.selectbox(
            "Selecciona la Variable Numérica:",
            options=numerical_vars,
            key="univariate_num_var"
        )

        if selected_var:
            st.write(f"### Distribución de `{selected_var}`")

            # Histograma
            fig_hist = px.histogram(df1, x=selected_var, nbins=30, title=f'Histograma de `{selected_var}`')
            st.plotly_chart(fig_hist)

            # Boxplot
            fig_box = px.box(df1, y=selected_var, title=f'Boxplot de `{selected_var}`')
            st.plotly_chart(fig_box)

            # Estadísticas descriptivas
            st.write("### Estadísticas Descriptivas")
            st.write(df1[selected_var].describe())

#################################################################

# 5. FRAME: Regresión Lineal
elif Variables == "Regresión Simple":
    st.title("Regresión Lineal Simple")
    st.write("""
        En esta sección puedes realizar un análisis de regresión lineal entre una variable dependiente y una independiente.
        Selecciona las variables que deseas analizar.
    """)

    # Seleccionar variables independientes y dependientes
    Var_Independiente = st.selectbox(
        "Selecciona la Variable Independiente (X):",
        options=numeric_cols1,
        key="regresion_x"
    )
    Var_Dependiente = st.selectbox(
        "Selecciona la Variable Dependiente (Y):",
        options=numeric_cols1,
        key="regresion_y"
    )

    if Var_Independiente and Var_Dependiente:
        st.write(f"### Análisis de Regresión Lineal: {Var_Independiente} vs {Var_Dependiente}")

        # Declarar las variables X y Y
        X = data_evaluar[[Var_Independiente]]
        y = data_evaluar[Var_Dependiente]

        # Dividir los datos en entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Definir el modelo de regresión lineal
        modelo_rl = LinearRegression()

        # Entrenar el modelo
        modelo_rl.fit(X_train, y_train)

        # Realizar predicciones
        y_pred = modelo_rl.predict(X_test)

        # Mostrar los coeficientes de regresión
        st.write(f"**Coeficiente**: {modelo_rl.coef_[0]:.4f}")
        st.write(f"**Intercepto**: {modelo_rl.intercept_:.4f}")

        # Calcular el R^2 del modelo
        r2 = r2_score(y_test, y_pred)
        st.write(f"**R² del modelo**: {r2:.4f}")

        # Visualización del modelo
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.scatter(X_test, y_test, color="blue", label="Datos Reales")
        ax.plot(X_test, y_pred, color="red", label="Línea de Regresión")
        ax.set_xlabel(Var_Independiente)
        ax.set_ylabel(Var_Dependiente)
        ax.legend()
        st.pyplot(fig)

#################################################################

# 6. FRAME: Regresión Múltiple
elif Variables == "Regresión Múltiple":
    st.title("Regresión Múltiple")
    st.write("""
        En esta sección puedes realizar un análisis de regresión múltiple.
        Selecciona las variables dependientes e independientes que deseas analizar.
    """)

    # Selección de la variable dependiente
    Var_Dependiente = st.selectbox(
        "Selecciona la Variable Dependiente (Y):",
        options=numeric_cols1,
        key="multiple_regression_y"
    )

    # Selección de múltiples variables independientes
    Vars_Independientes = st.multiselect(
        "Selecciona las Variables Independientes (X):",
        options=numeric_cols1,
        key="multiple_regression_x"
    )

    if Var_Dependiente and Vars_Independientes:
        st.write(f"### Análisis de Regresión Múltiple: {', '.join(Vars_Independientes)} vs {Var_Dependiente}")

        # Declarar las variables X e Y
        X = data_evaluar[Vars_Independientes]
        y = data_evaluar[Var_Dependiente]

        # Dividir los datos en entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Definir el modelo de regresión lineal múltiple
        modelo_rm = LinearRegression()

        # Entrenar el modelo
        modelo_rm.fit(X_train, y_train)

        # Realizar predicciones
        y_pred = modelo_rm.predict(X_test)

        # Mostrar los coeficientes de regresión
        st.subheader("Coeficientes del Modelo")
        coef_df = pd.DataFrame(modelo_rm.coef_, Vars_Independientes, columns=["Coeficiente"])
        st.write(coef_df)

        # Mostrar el intercepto
        st.write(f"**Intercepto**: {modelo_rm.intercept_:.4f}")

        # Calcular el R² del modelo
        r2 = r2_score(y_test, y_pred)
        st.write(f"**R² del modelo**: {r2:.4f}")

        # Visualización de los resultados
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.scatter(y_test, y_pred, color="blue")
        ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2, color="red")
        ax.set_xlabel("Valores Reales")
        ax.set_ylabel("Valores Predichos")
        st.pyplot(fig)

        st.write("""
            **Interpretación:**
            - Los puntos cercanos a la línea roja representan buenas predicciones.
            - El coeficiente de determinación (R²) mide qué tan bien las variables independientes explican la variabilidad en la variable dependiente.
        """)

#################################################################

# 4. FRAME: Regresión No Lineal
elif Variables == "Regresión No Lineal":
    st.title('Regresión No Lineal')
    st.write('Selecciona una o más variables para analizar su mejor modelo de regresión no lineal.')

    # Generamos un cuadro de selección múltiple (Multiselect) en la barra lateral para elegir las variables a graficar
    Vars_num_selected = st.sidebar.multiselect(
        label="Variable a Graficar",
        options=[
            'host_response_rate',
            'host_acceptance_rate',
            'host_total_listings_count',
            'accommodates',
            'reviews_per_month',
            'price'
        ],
        key="variables_graficar_no_lineal"
    )

    if Vars_num_selected:
        for var in Vars_num_selected:
            #obtener las principales variables correlacionadas
            a = corr_factors1.sort_values(by=var, ascending=False).head(7)
            a = a[1:].head()
            st.write(f'Tabla de Correlación para {var}')
            st.write(a)
            # Variable Independiente con mayor correlación
            a1 = a.index[0]
            st.write('Variable Independiente:', a1)

            # geclarar variables independientes y dependientes para la regresión
            vars_indep = data_evaluar[a1]
            var_dep = data_evaluar[var]

            #generar un cuadro de selección múltiple para elegir el tipo de modelo con clave única
            Vars_num_selected2 = st.sidebar.multiselect(
                label="Selecciona el Mejor Modelo:",
                options=[
                    'Modelo Exponencial',
                    'Modelo Cuadrático',
                    'Modelo Cociente entre Polinomios'
                ],
                key=f"modelo_select_{var}"
            )

            #aplicar el modelo seleccionado
            if 'Modelo Exponencial' in Vars_num_selected2:
                try:
                    def func_exponencial(x, a, b, c):
                        return a * np.exp(-b * x) + c

                    # Ajustar los parámetros de la función
                    parametros, covs = curve_fit(func_exponencial, vars_indep, var_dep)

                    # Realizar predicciones
                    y_pred = func_exponencial(vars_indep, *parametros)

                    # Crear el gráfico usando Plotly Graph Objects
                    import plotly.graph_objects as go

                    figure1 = go.Figure()

                    # Valores Reales
                    figure1.add_trace(go.Scatter(
                        x=vars_indep,
                        y=var_dep,
                        mode='markers',
                        name='Valores Reales',
                        marker=dict(color='green')
                    ))

                    # Valores Predichos
                    figure1.add_trace(go.Scatter(
                        x=vars_indep,
                        y=y_pred,
                        mode='markers',
                        name='Valores Predichos',
                        marker=dict(color='blue')
                    ))

                    # Actualizar el diseño del gráfico
                    figure1.update_layout(
                        title=f"Modelo Exponencial para {var}",
                        xaxis_title=a1,
                        yaxis_title=var,
                        width=1000,
                        height=500
                    )

                    # Mostrar el gráfico en Streamlit
                    st.plotly_chart(figure1)

                except Exception as e:
                    st.error(f"Ocurrió un error: {e}")

            if 'Modelo Cuadrático' in Vars_num_selected2:
                try:
                    def func_cuadratica(x, a, b, c):
                        return a * np.square(x) + b * x + c

                    # Ajustar los parámetros de la función
                    parametros, covs = curve_fit(func_cuadratica, vars_indep, var_dep)

                    # Realizar predicciones
                    y_pred = func_cuadratica(vars_indep, *parametros)

                    # Crear el gráfico usando Plotly Graph Objects
                    import plotly.graph_objects as go

                    figure2 = go.Figure()

                    # Trazas de Valores Reales
                    figure2.add_trace(go.Scatter(
                        x=vars_indep,
                        y=var_dep,
                        mode='markers',
                        name='Valores Reales',
                        marker=dict(color='green')
                    ))

                    # Trazas de Valores Predichos
                    figure2.add_trace(go.Scatter(
                        x=vars_indep,
                        y=y_pred,
                        mode='markers',
                        name='Valores Predichos',
                        marker=dict(color='blue')
                    ))

                    # Actualizar el diseño del gráfico
                    figure2.update_layout(
                        title=f"Modelo Cuadrático para {var}",
                        xaxis_title=a1,
                        yaxis_title=var,
                        width=1000,
                        height=500
                    )

                    # Mostrar el gráfico en Streamlit
                    st.plotly_chart(figure2)

                except Exception as e:
                    st.error(f"Ocurrió un error: {e}")

            if 'Modelo Cociente entre Polinomios' in Vars_num_selected2:
                try:
                    def func_cociente(x, a, b, c):
                        return (a * x**2 + b) / (c * x)

                    # Ajustar los parámetros de la función
                    parametros, covs = curve_fit(func_cociente, vars_indep, var_dep)

                    # Realizar predicciones
                    y_pred = func_cociente(vars_indep, *parametros)

                    # Crear el gráfico usando Plotly Graph Objects
                    import plotly.graph_objects as go

                    figure3 = go.Figure()

                    # Trazas de Valores Reales
                    figure3.add_trace(go.Scatter(
                        x=vars_indep,
                        y=var_dep,
                        mode='markers',
                        name='Valores Reales',
                        marker=dict(color='green')
                    ))

                    # Trazas de Valores Predichos
                    figure3.add_trace(go.Scatter(
                        x=vars_indep,
                        y=y_pred,
                        mode='markers',
                        name='Valores Predichos',
                        marker=dict(color='blue')
                    ))

                    # Actualizar el diseño del gráfico
                    figure3.update_layout(
                        title=f"Modelo Cociente entre Polinomios para {var}",
                        xaxis_title=a1,
                        yaxis_title=var,
                        width=1000,
                        height=500
                    )

                    # Mostrar el gráfico en Streamlit
                    st.plotly_chart(figure3)

                except Exception as e:
                    st.error(f"Ocurrió un error: {e}")

#################################################################

# 7. FRAME: Regresión Logística
elif Variables == "Regresión Logística":
    st.title("Regresión Logística")
    st.write("""
        En esta sección puedes realizar un análisis de regresión logística para predecir si las 'review_scores_rating' son mayores a 4.9.
    """)

    # Rellenamos valores nulos
    df_clean = df1.fillna(method='bfill').fillna(method='ffill')

    # Reemplazamos nuestra variable dependiente por valores binarios (1 si es mayor a 4.9, 0 en caso contrario)
    df_clean['review_scores_rating'] = (df_clean['review_scores_rating'] > 4.9).astype(int)

    # Declaramos las variables independientes
    Vars_Indep = df_clean[['number_of_reviews_ltm', 'review_scores_accuracy', 'review_scores_cleanliness',
                           'review_scores_checkin', 'review_scores_communication',
                           'review_scores_location', 'review_scores_value',
                           'calculated_host_listings_count', 'reviews_per_month']]

    # Variable dependiente
    Var_Dep = df_clean['review_scores_rating']

    # División de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(Vars_Indep, Var_Dep, test_size=0.3, random_state=42)

    # Escalar los datos
    escalar = StandardScaler()
    X_train = escalar.fit_transform(X_train)
    X_test = escalar.transform(X_test)

    # Definimos y entrenamos el modelo de regresión logística
    modelo_rl = LogisticRegression()
    modelo_rl.fit(X_train, y_train)

    # Realizamos predicciones
    y_pred = modelo_rl.predict(X_test)

    # Mostramos la matriz de confusión
    st.subheader("Matriz de Confusión")
    matriz_confusion = confusion_matrix(y_test, y_pred)
    st.write(matriz_confusion)

    # Calculamos y mostramos precisión, exactitud y sensibilidad
    precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
    exactitud = accuracy_score(y_test, y_pred)
    sensibilidad = recall_score(y_test, y_pred, average='binary', pos_label=1)

    st.subheader("Métricas del Modelo")
    st.write(f"**Precisión del modelo**: {precision:.2f}")
    st.write(f"**Exactitud del modelo**: {exactitud:.2f}")
    st.write(f"**Sensibilidad del modelo**: {sensibilidad:.2f}")


#################################################################

# 8. FRAME: ANOVA
elif Variables == "ANOVA":
    st.title("ANOVA")
    st.write("""
        En esta sección puedes realizar un análisis de varianza (ANOVA) para comparar las medias de una variable numérica entre diferentes grupos de una variable categórica.
    """)

    # Identificar variables categóricas y numéricas disponibles
    categorical_vars = [
        'instant_bookable',
        'room_type',
        'property_type',
        'neighbourhood',
        'has_availability'
        # Añade más variables categóricas aquí
    ]
    numerical_vars = [
        'review_scores_rating',
        'price',
        'minimum_nights',
        'number_of_reviews',
        'availability_365',
        'accommodates',
        'bedrooms',
        'beds'

    ]

    # Widgets para seleccionar las variables
    st.subheader("Selecciona las Variables para ANOVA")

    # Selectbox para seleccionar la variable categórica
    cat_var = st.selectbox(
        label="Selecciona la Variable Categórica",
        options=categorical_vars,
        key="anova_cat_var"  # Clave única para evitar duplicación de IDs
    )

    # Selectbox para seleccionar la variable numérica
    num_var = st.selectbox(
        label="Selecciona la Variable Numérica",
        options=numerical_vars,
        key="anova_num_var"  # Clave única para evitar duplicación de IDs
    )

    # Botón para realizar el análisis de ANOVA
    if st.button("Realizar ANOVA", key="anova_button"):
        st.write("### Preparando Datos...")

        # Crear una copia limpia del DataFrame para evitar modificaciones en el original
        df_clean = df1.copy()

        # Rellenar valores nulos
        df_clean = df_clean.fillna(method='bfill')
        df_clean = df_clean.fillna(method='ffill')

        # Verificar que las variables seleccionadas existen en el DataFrame
        if cat_var not in df_clean.columns or num_var not in df_clean.columns:
            st.error("Las variables seleccionadas no existen en el DataFrame.")
        else:
            # Verificar que la variable categórica tenga al menos dos grupos
            grupos = df_clean[cat_var].unique()
            if len(grupos) < 2:
                st.error("La variable categórica seleccionada debe tener al menos dos grupos para realizar ANOVA.")
            else:
                st.write(f"### Boxplot de `{num_var}` por `{cat_var}`")

                # Crear y mostrar el boxplot utilizando Seaborn
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.boxplot(x=cat_var, y=num_var, data=df_clean, palette="Set2")
                plt.title(f'Boxplot de {num_var} por {cat_var}')
                st.pyplot(fig)

                st.write("### Realizando ANOVA...")

                # Definir los grupos para ANOVA
                group_data = [df_clean[num_var][df_clean[cat_var] == grp] for grp in grupos]

                # Realizar el análisis de ANOVA
                anova_result = stats.f_oneway(*group_data)

                # Mostrar los resultados de ANOVA
                st.write("#### Resultado de ANOVA:")
                st.write(f"**Estadístico F:** {anova_result.statistic:.4f}")
                st.write(f"**Valor p:** {anova_result.pvalue:.4f}")

                if anova_result.pvalue < 0.05:
                    st.success("El valor p es menor que 0.05. Se rechaza la hipótesis nula, lo que implica que hay diferencias significativas entre las medias de los grupos.")
                else:
                    st.warning("El valor p es mayor o igual que 0.05. No se rechaza la hipótesis nula, lo que implica que no hay diferencias significativas entre las medias de los grupos.")

                st.write("### Tabla de ANOVA:")

                # Crear el modelo utilizando statsmodels
                formula = f'{num_var} ~ C({cat_var})'
                model = ols(formula, data=df_clean).fit()
                anova_table = sm.stats.anova_lm(model, typ=2)

                # Mostrar la tabla de ANOVA
                st.write(anova_table)

                st.write("""
                    **Interpretación:**
                    - **sum_sq:** Suma de cuadrados.
                    - **df:** Grados de libertad.
                    - **F:** Estadístico F.
                    - **PR(>F):** Valor p asociado al estadístico F.
                """)

                st.write("### Conclusión:")
                if anova_result.pvalue < 0.05:
                    st.write(f"Existe una diferencia significativa en las medias de `{num_var}` entre los diferentes grupos de `{cat_var}`.")
                else:
                    st.write(f"No existe una diferencia significativa en las medias de `{num_var}` entre los diferentes grupos de `{cat_var}`.")

                st.write("""
                    **Nota:** Para realizar un análisis MANOVA (Múltiples Variables ANOVA), selecciona múltiples variables numéricas y ajusta el modelo en consecuencia.
                """)

################################################################################

    #Customización del panel general
st.markdown(
   """
    <style>
    .stApp {
        background-color: gray; /* Cambia el color aquí */
    }
    h1 {
        font-size: 40px;
        color: green; /* Cambia el color aquí */
        font-weight: bold;
        text-align: left; /* Alinear a la izquierda el texto */
        }
    h2 {
        font-size: 24px;
        color: red; /* Cambia el color aquí */
        font-weight: bold;
        text-align: left; /* Alinear a la izquierda el texto */
        }
    h3 {
        font-size: 12px;
        color: blue; /* Cambia el color aquí */
        font-weight: bold;
        text-align: left; /* Alinear a la izquierda el texto */
        }
    """,
    unsafe_allow_html=True
)


############################################################################################
#Customización de la sidebar
st.sidebar.markdown(
    """
    <style>
    .stSidebar {
        background-color: black; /* Cambia el color aquí */
    }
    .stSidebar h1 {
        font-size: 30px;
        color: green; /* Cambia el color aquí */
        font-weight: bold;
        text-align: center; /* Centrar el texto */
    }
    .stSidebar h2 {
        font-size: 24px;
        color: white; /* Cambia el color aquí */
        font-weight: bold;
        text-align: left; /* Alinear a la izquierda el texto */
    }
    .stSidebar h3 {
        font-size: 18px;
        color: white; /* Cambia el color aquí */
        font-weight: bold;
        text-align: left; /* Alinear a la izquierda el texto */
    }
    .stButton > button {
        background-color: red; /* Cambia el color de fondo */
        color: white; /* Cambia el color del texto */
        padding: 10px 20px; /* Añade algo de padding */
        border: none; /* Sin bordes */
        border-radius: 5px; /* Bordes redondeados */
        cursor: pointer; /* Cambia el cursor */
    }
    .stButton > button:hover {
        background-color: green; /* Cambia el color al pasar el ratón */
        color: white; /* Cambia el color del texto */
    }
    </style>
    """,
    unsafe_allow_html=True
)
################################################################################


Overwriting app.py


In [10]:
!streamlit run app.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.16.154.21



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.154.21:8501

your url is: https://four-groups-leave.loca.lt
/content/app.py:45: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_copia['room_type'] = data_copia['room_type'].replace({
2024-10-18 22:51:50.968 Serialization of dataframe to Arrow table was unsuccessful due to: ("Could not convert 'Alejandra' with type str: tried to convert to int64", 'Conversion failed for column host_name with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
2024-10-18 22:52:33.149 Serialization of dataframe to Arrow table was unsuccessful due to: ("Could 